In [7]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays
import sqlite3  # Para conectarse a la base de datos SQLite
from datetime import datetime, timedelta  # Para obtener la fecha y hora actual
import os  # Para manejar el sistema de archivos
import joblib  # Para guardar y cargar modelos entrenados
import time  # Para medir el tiempo de procesamiento

# Librerías para construir pipelines y preprocesamiento
from sklearn.pipeline import Pipeline  # Para crear pipelines que incluyan pasos de preprocesamiento y modelo
from sklearn.preprocessing import StandardScaler  # Para escalar variables numéricas
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a diferentes tipos de columnas

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Algoritmo de clasificación basado en árboles y Stacking
from xgboost import XGBClassifier  # Algoritmo XGBoost, un modelo de boosting eficiente
from catboost import CatBoostClassifier  # Algoritmo CatBoost, un modelo de boosting que maneja variables categóricas internamente

# Librerías para validación cruzada y evaluación de modelos
from sklearn.model_selection import cross_validate, StratifiedKFold  # Validación cruzada y división estratificada de los datos
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # Métricas para evaluar el rendimiento de los modelos

# Configuración cuaderno Jupyter: Mostrar todas las columnas y ajustar ancho de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Función para conectar a la base de datos
def connect_db():
    try:
        conn = sqlite3.connect('../data/database/airline_satisfaction.db')
        return conn
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

# Función para crear la tabla si no existe
def create_table():
    conn = connect_db()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS modelos_entrenados (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                modelo TEXT,
                accuracy_mean REAL,
                precision_mean REAL,
                recall_mean REAL,
                f1_mean REAL,
                roc_auc_mean REAL,
                tiempo_procesamiento TEXT,
                archivo_modelo TEXT,
                n_estimators_catboost INTEGER,
                depth_catboost INTEGER,
                learning_rate_catboost REAL,
                n_estimators_rf INTEGER,
                n_estimators_xgb INTEGER,
                depth_xgb INTEGER,
                learning_rate_xgb REAL,
                fecha_entrenamiento TEXT
            )
            ''')
            conn.commit()
            conn.close()
            print("Tabla 'modelos_entrenados' creada o ya existente.")
        except Exception as e:
            print(f"Error al crear la tabla: {e}")

# Función para insertar datos en la nueva tabla
def insertar_modelo(modelo_nombre, accuracy, precision, recall, f1, roc_auc, tiempo, archivo, 
                    n_estimators_catboost=None, depth_catboost=None, learning_rate_catboost=None, 
                    n_estimators_rf=None, n_estimators_xgb=None, depth_xgb=None, learning_rate_xgb=None):
    conn = connect_db()
    if conn:
        try:
            cursor = conn.cursor()
            fecha_actual = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            cursor.execute('''
                INSERT INTO modelos_entrenados 
                (modelo, accuracy_mean, precision_mean, recall_mean, f1_mean, roc_auc_mean, 
                tiempo_procesamiento, archivo_modelo, n_estimators_catboost, depth_catboost, 
                learning_rate_catboost, n_estimators_rf, n_estimators_xgb, depth_xgb, 
                learning_rate_xgb, fecha_entrenamiento)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (modelo_nombre, accuracy, precision, recall, f1, roc_auc, tiempo, archivo, 
                  n_estimators_catboost, depth_catboost, learning_rate_catboost, n_estimators_rf, 
                  n_estimators_xgb, depth_xgb, learning_rate_xgb, fecha_actual))
            conn.commit()
            conn.close()
        except Exception as e:
            print(f"Error al insertar datos: {e}")

# Crear la tabla (si no existe) antes de comenzar la evaluación de modelos
create_table()

# Paso 1: Cargar los datos (CSV limpio)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

# Paso 2: Codificar manualmente las variables categóricas usando `map`
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

categorical_mappings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

for col, mapping in categorical_mappings.items():
    X[col] = X[col].map(mapping)

ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)
    ]
)

# Paso 4: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False))
])

stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),
        ('xgboost', pipeline_xgb),
        ('random_forest', pipeline_rf)
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5
)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_readable = str(timedelta(seconds=elapsed_time))
    
    return cv_results, elapsed_time, elapsed_time_readable

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

resultados = []

modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    cv_resultados, tiempo, tiempo_legible = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'tiempo_procesamiento_legible': tiempo_legible,
        'modelo_pipeline': modelo
    })

df_resultados = pd.DataFrame(resultados)

df_vista_consola = df_resultados.drop(columns=['modelo_pipeline'])

print("\nResultados finales:")
print(tabulate(df_vista_consola, headers='keys', tablefmt='psql'))

output_csv = '../data/modelos_entrenamiento/resultados_modelos_completos.csv'
df_resultados.to_csv(output_csv, index=False)
print(f"\nLos resultados completos se han guardado en el archivo: {output_csv}")

mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()]

print(f"\nEl mejor modelo ha sido: {mejor_modelo['modelo']} debido a su mejor rendimiento en accuracy de {mejor_modelo['accuracy_mean']:.4f}")
print(f"\nEntrenando el mejor modelo: {mejor_modelo['modelo']} con todos los datos...")

modelo_final = mejor_modelo['modelo_pipeline']
modelo_final.fit(X, y)

accuracy = mejor_modelo['accuracy_mean']
precision = mejor_modelo['precision_mean']
recall = mejor_modelo['recall_mean']
f1 = mejor_modelo['f1_mean']
roc_auc = mejor_modelo['roc_auc_mean']
tiempo_procesamiento = mejor_modelo['tiempo_procesamiento_legible']
archivo_modelo = ""

def guardar_modelo_con_nombre(modelo, nombre_modelo):
    output_dir = '../data/modelos_entrenamiento/'
    os.makedirs(output_dir, exist_ok=True)
    
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    if nombre_modelo == 'CatBoost':
        archivo_modelo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.cbm"
        modelo.named_steps['model'].save_model(os.path.join(output_dir, archivo_modelo))
        print(f"Modelo CatBoost guardado exitosamente con el nombre: {archivo_modelo}")
    else:
        archivo_modelo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl"
        joblib.dump(modelo, os.path.join(output_dir, archivo_modelo))
        print(f"Modelo guardado exitosamente con el nombre: {archivo_modelo}")
    
    return archivo_modelo

archivo_modelo = guardar_modelo_con_nombre(modelo_final, mejor_modelo['modelo'])

n_estimators_catboost = None
depth_catboost = None
learning_rate_catboost = None
n_estimators_rf = None
n_estimators_xgb = None
depth_xgb = None
learning_rate_xgb = None

if mejor_modelo['modelo'] == 'CatBoost':
    n_estimators_catboost = modelo_final.named_steps['model'].get_params().get('iterations')
    depth_catboost = modelo_final.named_steps['model'].get_params().get('depth')
    learning_rate_catboost = modelo_final.named_steps['model'].get_params().get('learning_rate')
elif mejor_modelo['modelo'] == 'Random Forest':
    n_estimators_rf = modelo_final.named_steps['model'].get_params().get('n_estimators')
elif mejor_modelo['modelo'] == 'XGBoost':
    n_estimators_xgb = modelo_final.named_steps['model'].get_params().get('n_estimators')
    depth_xgb = modelo_final.named_steps['model'].get_params().get('max_depth')
    learning_rate_xgb = modelo_final.named_steps['model'].get_params().get('learning_rate')

insertar_modelo(mejor_modelo['modelo'], accuracy, precision, recall, f1, roc_auc, tiempo_procesamiento, archivo_modelo, 
                n_estimators_catboost, depth_catboost, learning_rate_catboost, 
                n_estimators_rf, n_estimators_xgb, depth_xgb, learning_rate_xgb)

print(f"Modelo {mejor_modelo['modelo']} y sus parámetros han sido almacenados en la base de datos.")


Tabla 'modelos_entrenados' creada o ya existente.

Evaluando Random Forest...

Evaluando XGBoost...

Evaluando CatBoost...

Evaluando Stacking...
